In [2]:
from theano.sandbox import cuda

In [3]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.
C:\Users\Shikhar\Anaconda2\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
#path = "data/ml-20m/"
path = "data/ml-small/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size=64

In [5]:
ratings = pd.read_csv(path+'ratings.csv')
ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [6]:
users = ratings.userId.unique()
movies = ratings.movieId.unique()

In [7]:
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

In [8]:
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])

In [9]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
n_users, n_movies

(671, 9066)

In [10]:
n_factors = 50

In [11]:
np.random.seed = 42

In [12]:
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]


In [13]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg))(inp)

In [14]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-4)

In [15]:

x = merge([u, m], mode='concat')
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(70, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(1)(x)
nn = Model([user_in, movie_in], x)
nn.compile(Adam(0.001), loss='mse')

In [16]:
nn.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=8, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 79995 samples, validate on 20009 samples
Epoch 1/8
79995/79995 [==============================] - 26s - loss: 2.3422 - val_loss: 0.9402
Epoch 2/8
79995/79995 [==============================] - 25s - loss: 1.4135 - val_loss: 0.8712
Epoch 3/8
79995/79995 [==============================] - 24s - loss: 1.2012 - val_loss: 0.8599
Epoch 4/8
79995/79995 [==============================] - 22s - loss: 1.0262 - val_loss: 0.8583
Epoch 5/8
79995/79995 [==============================] - 26s - loss: 0.9114 - val_loss: 0.8436
Epoch 6/8
79995/79995 [==============================] - 27s - loss: 0.8488 - val_loss: 0.8329
Epoch 7/8
79995/79995 [==============================] - 28s - loss: 0.8208 - val_loss: 0.8351
Epoch 8/8
79995/79995 [==============================] - 24s - loss: 0.8059 - val_loss: 0.8401


In [30]:
nn.save_weights(model_path+'neural0_8030_0_8564')
nn.load_weights(model_path+'neural0_8030_0_8564')

In [17]:
nn.predict([np.array([1]), np.array([31])])


array([[3.6874]], dtype=float32)

2